# Preparation

In [1]:
seed_value= 42

import random
random.seed(seed_value)

import numpy as np
np.random.seed(seed_value)

import tensorflow as tf
tf.random.set_seed(seed_value)
tf.keras.utils.set_random_seed(seed_value)

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.9.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
num_epochs  = 100
batch_size  = 16
num_classes = 10
shape       = (28, 28, 1)
lr          = 0.0003
opt         = keras.optimizers.Adamax(learning_rate=lr)
los         = keras.losses.CategoricalCrossentropy()
mtr         = ["accuracy"]

# Dataset

In [5]:
def prepare_data(main_path, test_path, validation_split=0.1):
    with np.load(main_path) as data:
        x_train, y_train = data['images'], data['labels']
        
    with np.load(test_path) as data:
        x_test, y_test = data['x_test'], data['y_test']
        
    # Normalize and reshape the data
    x_train = x_train.astype("float32") / 255.0
    x_test = x_test.astype("float32") / 255.0
    
    x_train = np.reshape(x_train, (-1, 28, 28, 1))
    x_test = np.reshape(x_test, (-1, 28, 28, 1))
    
    y_test = keras.utils.to_categorical(y_test, 10)
    
    # Split the training data into train and validation sets
    val_size = int(len(x_train) * validation_split)
    x_val, y_val = x_train[:val_size], y_train[:val_size]
    x_train, y_train = x_train[val_size:], y_train[val_size:]
    
    # Create TensorFlow datasets
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    
    # Shuffle and batch the datasets
    train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)
    val_dataset = val_dataset.shuffle(buffer_size=1024).batch(batch_size)
    test_dataset = test_dataset.shuffle(buffer_size=1024).batch(batch_size)
    
    # Print the shapes
    print(f"Shape of training images: {x_train.shape}")
    print(f"Shape of validation images: {x_val.shape}")
    print(f"Shape of testing images: {x_test.shape}")
    
    return train_dataset, val_dataset, test_dataset

    
train_dataset, val_dataset, test_dataset = prepare_data(main_path="../Dataset/NoReg_Extra_6.npz",
                                                        test_path="../Dataset/mnist.npz")    

Shape of training images: (54000, 28, 28, 1)
Shape of validation images: (6000, 28, 28, 1)
Shape of testing images: (10000, 28, 28, 1)


# Model

In [6]:
model = keras.Sequential([keras.layers.InputLayer(shape),
                          layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same"),
                          layers.LeakyReLU(alpha=0.2),
                          layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
                          layers.LeakyReLU(alpha=0.2),
                          layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
                          layers.LeakyReLU(alpha=0.2),
                          layers.GlobalMaxPooling2D(),
                          layers.Dense(64, activation="relu"),
                          layers.Dense(num_classes, activation="softmax")],
                         name="discriminator")

model.compile(optimizer=opt, loss=los, metrics=mtr)
model.summary()

Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 14, 14, 32)        320       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 14, 14, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 64)          18496     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 7, 7, 64)          0         
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 4, 4, 128)         0         
                                                                 
 global_max_pooling2d (Globa  (None, 128)            

# Train

In [7]:
def callback():
  main_chk  = keras.callbacks.ModelCheckpoint(filepath="Checkpoints/Simple", monitor='val_loss', mode='min', verbose=0, save_best_only=True)
  early_st  = keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=30, verbose=0)
  rduce_lr  = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', factor=0.5, patience=5, verbose=1, min_lr=0.00001)

  return [main_chk, early_st, rduce_lr]

model.fit(train_dataset, validation_data=val_dataset, epochs=num_epochs, batch_size=batch_size, callbacks=callback())

Epoch 1/100
3367/3375 [============================>.] - ETA: 0s - loss: 0.1347 - accuracy: 0.9809

INFO:tensorflow:Assets written to: Checkpoints\Simple\assets


INFO:tensorflow:Assets written to: Checkpoints\Simple\assets


3375/3375 [==============================] - 11s 3ms/step - loss: 0.1343 - accuracy: 0.9809 - val_loss: 5.0524e-04 - val_accuracy: 1.0000 - lr: 3.0000e-04
Epoch 2/100
3355/3375 [============================>.] - ETA: 0s - loss: 1.9624e-04 - accuracy: 1.0000

INFO:tensorflow:Assets written to: Checkpoints\Simple\assets


...

INFO:tensorflow:Assets written to: Checkpoints\Simple\assets


3375/3375 [==============================] - 9s 3ms/step - loss: 7.0863e-10 - accuracy: 1.0000 - val_loss: 6.0002e-09 - val_accuracy: 1.0000 - lr: 3.7500e-05
Epoch 22/100
3373/3375 [============================>.] - ETA: 0s - loss: 6.7813e-10 - accuracy: 1.0000

INFO:tensorflow:Assets written to: Checkpoints\Simple\assets


INFO:tensorflow:Assets written to: Checkpoints\Simple\assets



Epoch 22: ReduceLROnPlateau reducing learning rate to 1.8750000890577212e-05.
3375/3375 [==============================] - 14s 4ms/step - loss: 6.7773e-10 - accuracy: 1.0000 - val_loss: 5.9008e-09 - val_accuracy: 1.0000 - lr: 3.7500e-05
Epoch 23/100
3375/3375 [==============================] - 13s 4ms/step - loss: 6.5124e-10 - accuracy: 1.0000 - val_loss: 7.8280e-09 - val_accuracy: 1.0000 - lr: 1.8750e-05
Epoch 24/100
3375/3375 [==============================] - 9s 3ms/step - loss: 6.0488e-10 - accuracy: 1.0000 - val_loss: 7.4306e-09 - val_accuracy: 1.0000 - lr: 1.8750e-05
Epoch 25/100
3375/3375 [==============================] - 9s 3ms/step - loss: 6.0488e-10 - accuracy: 1.0000 - val_loss: 7.3710e-09 - val_accuracy: 1.0000 - lr: 1.8750e-05
Epoch 26/100
3375/3375 [==============================] - 9s 3ms/step - loss: 5.7839e-10 - accuracy: 1.0000 - val_loss: 7.4505e-09 - val_accuracy: 1.0000 - lr: 1.8750e-05
Epoch 27/100
3371/3375 [============================>.] - ETA: 0s - loss: 5.6

# Evaluation

In [8]:
test_model = tf.keras.models.load_model("Checkpoints/Simple")
test_model.evaluate(test_dataset, verbose=1, batch_size=batch_size)

625/625 [==============================] - 1s 1ms/step - loss: 2.1881 - accuracy: 0.7193


[2.1881039142608643, 0.7192999720573425]